In [73]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '../input/herbarium-2022-fgvc9/train_images/'
test_dir = '../input/herbarium-2022-fgvc9/test_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

## import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '../input/herbarium-2022-fgvc9/train_images/'
test_dir = '../input/herbarium-2022-fgvc9/test_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

In [74]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
main_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
Never_used = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

main_df

,image_id,image_dir,category,genus
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,1
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,1
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,1
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,1
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,1
...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584


In [75]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
main_df['genus'] = main_df['genus'].map(genus_map)

##Create a family column in the datagframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
main_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in main_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        main_df.at[i, 'family'] = family

main_df

,image_id,image_dir,category,genus,family
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae


In [78]:
#Filter only the images of plants that are in the Poaceae family
main_df = main_df[main_df['family'].isin(['Poaceae', 'Fabaceae'])]
#Reset index
main_df = main_df.reset_index(drop=True)
main_df

,image_id,image_dir,category,genus,family
0,00045__001,../input/herbarium-2022-fgvc9/train_images/000...,45,Acaciella,Fabaceae
1,00045__002,../input/herbarium-2022-fgvc9/train_images/000...,45,Acaciella,Fabaceae
2,00045__003,../input/herbarium-2022-fgvc9/train_images/000...,45,Acaciella,Fabaceae
3,00045__005,../input/herbarium-2022-fgvc9/train_images/000...,45,Acaciella,Fabaceae
4,00045__007,../input/herbarium-2022-fgvc9/train_images/000...,45,Acaciella,Fabaceae
...,...,...,...,...,...
112694,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
112695,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
112696,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
112697,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae


In [67]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image

!pip install timm
import timm
!pip install fvcore

In [68]:
#main_df['genus'] = pd.factorize(main_df['genus'])[0]
main_df

,image_id,image_dir,category,genus,family
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae


In [53]:
grouped = main_df.groupby('genus')
# Split the dataset for each class separately
train_dfs = []
test_dfs = []

for name, group in grouped:
    # Split the group into train and test sets
    train, test = train_test_split(group, test_size=0.2, random_state=42)
    # Add the train and test dataframes to the respective lists
    train_dfs.append(train)
    test_dfs.append(test)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df_initial = pd.concat(train_dfs)
train_df_initial = train_df_initial.reset_index(drop=True)

test_df = pd.concat(test_dfs)
test_df = test_df.reset_index(drop=True)

In [54]:
grouped2 = train_df_initial.groupby('genus')
# Split the dataset for each class separately
train_dfs = []
val_dfs = []

for name, group in grouped2:
    # Split the group into train and test sets
    train, validation = train_test_split(group, test_size=0.2, random_state=42)
    # Add the train and test dataframes to the respective lists
    train_dfs.append(train)
    val_dfs.append(validation)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df = pd.concat(train_dfs)
train_df = train_df.reset_index(drop=True)

val_df = pd.concat(val_dfs)
val_df = val_df.reset_index(drop=True)

In [55]:
batch_size = 32
epochs = 3
IM_SIZE = 224
learning_rate = 1e-4

X_train, Y_train = train_df["image_dir"].values, train_df["genus"].values

X_val, Y_val = val_df["image_dir"].values, val_df["genus"].values

X_test, Y_test  = test_df["image_dir"].values, test_df["genus"].values

Transform = transforms.Compose([
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [56]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
        
        
trainset = GetData(X_train, Y_train, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_val, Y_val, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

testset = GetData(X_test, Y_test, Transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest101', pretrained=True)
#model = models.resnet152(pretrained=True)

Using cache found in /root/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [57]:
num_classes = train_df['genus'].nunique()
print(num_classes)

158


In [58]:
total_layers = len(list(model.parameters()))

for param in model.parameters():
    param.requires_grad = True
    
n_inputs = model.fc.in_features
last_layer = nn.Linear(n_inputs, num_classes)
model.fc = last_layer 
if torch.cuda.is_available():
    model.cuda()
print(model.fc.out_features)    

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

158


In [59]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    model.train()
    train_acc = 0.0
    train_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            y_true += labels.cpu().numpy().tolist()
            y_pred += output.argmax(dim=1).cpu().numpy().tolist()
            
    train_f1 = f1_score(y_true, y_pred, average=None)
    train_f1_avg = f1_score(y_true, y_pred, average='macro')
    return train_acc/len(trainloader), train_loss/len(trainloader), train_f1, train_f1_avg

In [60]:
def evaluate(valloader, model, criterion, device=torch.device("cpu")):
    model.eval()
    eval_acc = 0.0
    eval_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(valloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()
  
    eval_f1 = f1_score(y_true, y_pred, average=None)
    eval_f1_avg = f1_score(y_true, y_pred, average='macro')
    return eval_acc/len(valloader), eval_loss/len(valloader), eval_f1, eval_f1_avg

In [61]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores


for epoch in range(epochs):
    train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
    val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")
    
    print("F1 score per class (Train):")
    for i, f1 in enumerate(train_f1):
        print(f"Class {i}: {f1}")

    print("\nF1 score per class (Validation):") 
    for i, f1 in enumerate(eval_f1):
        print(f"Class {i}: {f1}")

    print("===="*8)

100%|██████████| 270/270 [03:55<00:00,  1.15it/s]


Epoch 1 | Train Acc: 51.38303756713867 | Train Loss: 1.9909052848815918 | Train F1 (Avg): 0.29115416129980626
	 Val Acc: 70.09424591064453 | Val Loss: 1.1056705713272095 | Val F1 (Avg): 0.5091626196785638
F1 score per class (Train):
Class 0: 0.49029622063329925
Class 1: 0.0
Class 2: 0.5071225071225072
Class 3: 0.0
Class 4: 0.4714285714285714
Class 5: 0.6223717409587889
Class 6: 0.4657534246575342
Class 7: 0.1694915254237288
Class 8: 0.4142857142857143
Class 9: 0.0
Class 10: 0.6712727272727274
Class 11: 0.2978723404255319
Class 12: 0.384
Class 13: 0.12244897959183675
Class 14: 0.3137254901960784
Class 15: 0.0
Class 16: 0.23809523809523808
Class 17: 0.4684684684684684
Class 18: 0.6254295532646048
Class 19: 0.5718533201189296
Class 20: 0.5903614457831325
Class 21: 0.6581439393939393
Class 22: 0.4509018036072144
Class 23: 0.1851851851851852
Class 24: 0.46511627906976744
Class 25: 0.0
Class 26: 0.08
Class 27: 0.4860557768924303
Class 28: 0.5350734094616639
Class 29: 0.0
Class 30: 0.39705882

100%|██████████| 270/270 [03:43<00:00,  1.21it/s]


Epoch 2 | Train Acc: 79.36174011230469 | Train Loss: 0.7313400506973267 | Train F1 (Avg): 0.6537237088048399
	 Val Acc: 79.10879516601562 | Val Loss: 0.7600000500679016 | Val F1 (Avg): 0.6592487015296279
F1 score per class (Train):
Class 0: 0.753315649867374
Class 1: 0.0
Class 2: 0.825974025974026
Class 3: 0.0
Class 4: 0.8586387434554975
Class 5: 0.8422791081750619
Class 6: 0.8216216216216217
Class 7: 0.6
Class 8: 0.8554913294797688
Class 9: 0.0
Class 10: 0.8946540880503145
Class 11: 0.7605633802816901
Class 12: 0.7586206896551726
Class 13: 0.6197183098591549
Class 14: 0.7094801223241591
Class 15: 0.0
Class 16: 0.73015873015873
Class 17: 0.8244274809160306
Class 18: 0.8429487179487181
Class 19: 0.8483853311439519
Class 20: 0.8888888888888888
Class 21: 0.9037037037037037
Class 22: 0.7449856733524356
Class 23: 0.7368421052631579
Class 24: 0.8048452220726783
Class 25: 0.7741935483870968
Class 26: 0.5569620253164557
Class 27: 0.8698412698412699
Class 28: 0.802962962962963
Class 29: 0.0
Cla

100%|██████████| 270/270 [03:08<00:00,  1.43it/s]

Epoch 3 | Train Acc: 88.64115142822266 | Train Loss: 0.3904830813407898 | Train F1 (Avg): 0.8109053784383167
	 Val Acc: 80.30919647216797 | Val Loss: 0.7127684354782104 | Val F1 (Avg): 0.6738325549693375
F1 score per class (Train):
Class 0: 0.8327402135231318
Class 1: 0.6666666666666666
Class 2: 0.9020618556701031
Class 3: 0.1818181818181818
Class 4: 0.907103825136612
Class 5: 0.9342327150084317
Class 6: 0.9281767955801105
Class 7: 0.7674418604651162
Class 8: 0.8972972972972972
Class 9: 0.4
Class 10: 0.9454834858734579
Class 11: 0.8333333333333333
Class 12: 0.8888888888888888
Class 13: 0.7341772151898734
Class 14: 0.8614457831325302
Class 15: 0.0
Class 16: 0.787878787878788
Class 17: 0.8507462686567164
Class 18: 0.9345794392523364
Class 19: 0.9234111895708853
Class 20: 0.964824120603015
Class 21: 0.9560148383677796
Class 22: 0.8599605522682445
Class 23: 0.8723404255319149
Class 24: 0.8810810810810812
Class 25: 0.8235294117647058
Class 26: 0.7710843373493976
Class 27: 0.9274447949526814